In [45]:
# =====================================
# Inverse Modeling: Gain + UGF + PM → W X Y
# =====================================

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from joblib import load, dump


In [46]:
df = pd.read_csv(r"E:\VIIT\TY\UST Analog Workflow Automation\Project\UST_Analog_automation\data\processed\Opam\merged_ml_ready.csv")

X = df[["Gain", "ugf", "pm"]]
y = df[["W", "X", "Y"]]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [47]:
inverse_rf = RandomForestRegressor(
    n_estimators=500,
    max_depth=None,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

inverse_rf.fit(X_train, y_train)
y_pred = inverse_rf.predict(X_test)


In [48]:
for i, param in enumerate(["W", "X", "Y"]):
    r2 = r2_score(y_test.iloc[:, i], y_pred[:, i])
    mae = mean_absolute_error(y_test.iloc[:, i], y_pred[:, i])
    print(f"{param}: R²={r2:.4f}, MAE={mae:.3e}")


W: R²=0.9976, MAE=3.703e-07
X: R²=0.8957, MAE=2.296e-07
Y: R²=0.8291, MAE=6.089e-07


In [49]:
# Load forward models
from joblib import load

gain_fwd = load(
    r"E:\VIIT\TY\UST Analog Workflow Automation\Project\UST_Analog_automation\models\Opam\rf_gain_forward.pkl"
)

ugf_fwd = load(
    r"E:\VIIT\TY\UST Analog Workflow Automation\Project\UST_Analog_automation\models\Opam\rf_ugf_forward.pkl"
)

pm_fwd = load(
    r"E:\VIIT\TY\UST Analog Workflow Automation\Project\UST_Analog_automation\models\Opam\rf_pm_forward.pkl"
)

# Take desired specs
desired_specs = X_test.iloc[:10]

# Inverse prediction
pred_geom = inverse_rf.predict(desired_specs)
pred_geom = pd.DataFrame(pred_geom, columns=["W","X","Y"])

# Forward re-evaluation
gain_check = gain_fwd.predict(pred_geom)
ugf_check  = ugf_fwd.predict(pred_geom)
pm_check   = pm_fwd.predict(pred_geom)

comparison = desired_specs.copy()
comparison["gain_pred"] = gain_check
comparison["ugf_pred"]  = ugf_check
comparison["pm_pred"]   = pm_check

comparison


,Gain,ugf,pm,gain_pred,ugf_pred,pm_pred
521,47.190974,1.422757e+08,67.070848,49.681749,148.197814,62.035972
737,38.223199,1.222478e+08,85.314740,38.202197,123.067042,85.016915
740,37.017741,1.305347e+08,83.320860,38.743767,129.643567,83.004345
660,43.350232,1.509840e+08,69.949025,43.455606,150.062493,70.231293
411,51.114375,1.435807e+08,57.982675,51.056934,142.963574,57.613152
678,45.849968,1.507100e+08,67.961213,45.969234,150.188276,69.545488
626,43.045284,1.288699e+08,78.032425,43.116992,126.285458,78.792707
513,48.532874,1.276102e+08,68.631349,48.497714,130.031065,67.943560
859,32.225828,1.004763e+08,98.106112,31.598315,93.255167,101.814177
136,55.754174,1.346590e+08,65.515047,55.750914,134.924128,65.481897


In [50]:
from joblib import dump
dump(inverse_rf, "inverse_rf.pkl")

['inverse_rf.pkl']

In [51]:
from joblib import load
import pandas as pd

inverse_rf = load(r"E:\VIIT\TY\UST Analog Workflow Automation\Project\UST_Analog_automation\models\Opam\inverse_rf.pkl")


In [52]:
user_input = pd.DataFrame([{
    "Gain": 56.0,
    "ugf": 135.0,
    "pm": 60.0
}])


In [53]:
predicted_wxy = inverse_rf.predict(user_input)

W, X, Y = predicted_wxy[0]

print(f"W = {W:.3e}")
print(f"X = {X:.3e}")
print(f"Y = {Y:.3e}")


W = 4.793e-05
X = 2.793e-06
Y = 9.464e-06


In [54]:
gain_pred = gain_fwd.predict(pd.DataFrame([[W,X,Y]], columns=["W","X","Y"]))
ugf_pred  = ugf_fwd.predict(pd.DataFrame([[W,X,Y]], columns=["W","X","Y"]))
pm_pred   = pm_fwd.predict(pd.DataFrame([[W,X,Y]], columns=["W","X","Y"]))

print("Validated Gain:", gain_pred[0])
print("Validated UGF :", ugf_pred[0])
print("Validated PM  :", pm_pred[0])


Validated Gain: 55.418438319432354
Validated UGF : 139.29063297807858
Validated PM  : 60.31410930772132
